# Introduction

# Setup


In [ ]:
import wandb
from lightning.pytorch.loggers import WandbLogger

wandb.login(key="")
wandb_logger = WandbLogger(project="experiment-tracking")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/maiko/.netrc
wandb: Currently logged in as: maikotrede (maikotrede-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### WandDB

In [ ]:

def init_wandb_run(project_name, run_name, config_dict):
    run = wandb.init(
        project=project_name,
        name=run_name,
        config=config_dict,
    )
    return run


#### Example Run
experiment_1_config = {
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 32,
    "optimizer": "adam"
}
run1 = init_wandb_run("experiment-tracking", "experiment_1", experiment_1_config)


# Preprocessing

In [1]:
from datasets import load_dataset

ds = load_dataset("tau/commonsense_qa")

README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

# Model

# Training

# Evaluation

# Interpretation